# Maps and data on maps visualisations 


Created on Sun Apr 21 18:33:02 2019
   @author: liubov
   @authors: Hugo, Prashant, open for collaborations


Project is inspired by Unicef school mapping project and 
Lecturers without borders project at CRI www.scied.network 

*"There are as many ways of drawing maps, as there are types of data"*, Christian Ruder



# Input data: 
we have N data-points on a world map, which come from the database (which is updated online e.g. verified users can add points on map).
The example files .csv 
<img src="map.png" alt="Drawing" style="width: 200px;"/>

As an example we put here the openstreetmap file from openstreet map project for mapping of educational institutions around the world.  The problem with it is that because of the sensitive data in the file we cannot make it visible for people, however we want the following output.

# Output: 
The unverified users cannot see the input datapoints (which is not shown for privacy reasons), but the map with colored countries, using https://www.d3-graph-gallery.com/bubblemap.html 
according to the input of database



### Plotting functions

We plot the world map using geopandas module.
Geopandas provides a high-level interface to the matplotlib library for making maps. Mapping shapes is as easy as using the plot() method on a GeoSeries or GeoDataFrame.

To read more about the cartopy feature go to https://geopandas.readthedocs.io/en/latest/mapping.html


In [1]:


import numpy as np
import pandas as pd
import geopandas as gpd

import csv
from collections import OrderedDict
import networkx as nx
from descartes import PolygonPatch


import matplotlib.pyplot as plt
import cartopy
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeat
import cartopy.crs as ccrs
from zipfile import ZipFile


#from mpl_toolkits.basemap import Basemap
#import matplotlib.pyplot as plt
##################################
"""
Function for analysis and visualisations of datapoints on a map
"""

# Example files put in pandas format with header
#airport_col = ['ID', 'Name', 'City', 'Country','IATA', 'ICAO', 'Lat', 'Long', 'Alt', 
#               'Timezone', 'DST', 'Tz database time zone', 'type', 'source']


def plot_on_map(lat,lon):
    '''
    basic function to plot points from data on a map
    the coordinates should be specified in a separate column
    '''
    
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_global()
    ax.coastlines()
    
    ll_to_pixel = ccrs.Geodetic()._as_mpl_transform(ax)
    
    # We need to call draw to ensure that the axes location has been defined
    # fully. 
    plt.draw()
    
    # Now lets figure out the pixel coordinate of the city.
    x1_pix, y1_pix = ll_to_pixel.transform_point([151.2111, -33.8600])
    x2_pix, y2_pix = ll_to_pixel.transform_point([101.2111, -33.8600])
    
    # We can even plot these pixel coordinates directly with matplotlib.
    plt.plot(x1_pix, y1_pix, 'ob', markersize=25, transform=None)
    plt.plot(x2_pix, y2_pix, 'ob', markersize=25, transform=None)
    
    #plt.savefig('figure_1.png', dpi=plt.gcf().get_dpi())
    plt.show()    


def arr_school(data): #calculate array of schools in the same country
    N_school_country = data['country'].nunique() #estimate number of various schools
    arr_schools = np.zeros(N_school_country)
    for ind in range(0,N_school_country):
        arr_schools[ind] = data[ind] #arr_schools[ind] = data.iloc('country' = country_i)
        

def plot_on_map(data,arr_schools):
    plot(basemap)
    plot(arr_schools,earth) #plot arr_schools for each country on earth 
    
    
def plot_basemap():
    print('plotting basemap')
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_global()
    ax.coastlines()
    
    ll_to_pixel = ccrs.Geodetic()._as_mpl_transform(ax)
    
    # We need to call draw to ensure that the axes location has been defined
    # fully. 
    plt.draw()  
    
    
def highlight_country(country):
    '''
    Now we highlight country on the world map
    As the example we put here 
    country = USA
    '''
    country = 'USA'
    ax = plt.axes(projection=ccrs.PlateCarree())
    #ax.add_feature(cartopy.feature.LAND)
    ax.add_feature(cartopy.feature.OCEAN)
    #ax.add_feature(cartopy.feature.COASTLINE)
    #ax.add_feature(cartopy.feature.BORDERS, linestyle='-', alpha=.5)
    #ax.add_feature(cartopy.feature.LAKES, alpha=0.95)
    #ax.add_feature(cartopy.feature.RIVERS)
    ax.set_extent([-150, 60, -25, 60])
    
    
    shpfilename = shpreader.natural_earth(resolution='110m',
                                          category='cultural',
                                          name='admin_0_countries')
    reader = shpreader.Reader(shpfilename)
    countries = reader.records()
    print('plotting basemap per country')
    
    for country in countries:
        if country.attributes['ADM0_A3'] == country:
            ax.add_geometries(country.geometry, ccrs.PlateCarree(),
                              facecolor=(0, 0, 1),
                              label=country.attributes['ADM0_A3'])
        else:
            ax.add_geometries(country.geometry, ccrs.PlateCarree(),
                              facecolor=(0, 1, 0),
                              label=country.attributes['ADM0_A3'])
    plt.show()   
    
    

def plot_country_color( axes, country_name, fcolor ):
    '''
    plot each country in specific color with provided axes
    '''
    nami = world[world.name == country_name]
    namigm = nami.__geo_interface__['features']  # geopandas's geo_interface
    namig0 = {'type': namigm[0]['geometry']['type'], \
              'coordinates': namigm[0]['geometry']['coordinates']}
    axes.add_patch(PolygonPatch( namig0, fc=fcolor, ec="black", alpha=0.85, zorder=2 ))

    
    
'''
First we plot the whole world map
'''
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

ax2 = world.plot( figsize=(8,4), edgecolor=u'gray', cmap='Set2' )
#another way to plot the world map
#world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

'''
Now we color the world with colors per country as number of datapoints
Warning: some errors due to missing values in some countries
'''
#for index, row in data.iterrows(): 
#    print (row["source_iso3"], row["estimated_trips"])
#    plot_country_color(ax2, row["source_iso3"], row["estimated_trips"])



'\nNow we color the world with colors per country as number of datapoints\nWarning: some errors due to missing values in some countries\n'

### Reading datafiles 

We take as an example several data files. 
1. Public_Schools_Points_2011-2012A.dbf (open data from google for schools)
2. randomized_edu_data_Nov_2018.csv (randomized school positions)


In [2]:
from dbfread import DBF
###########OPEN data FILE###################


#opening and printing data from dbf file 
print('reading schools from Public Schools Points')        
for record in DBF('Public_Schools_Points_2011-2012A.dbf'):
    print(record) #print each row
    
 

reading schools from Public Schools Points
OrderedDict([('ATS_CODE', '15K001 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'K'), ('BORONUM', 2), ('LOC_CODE', 'K001'), ('SCHOOLNAME', 'P.S. 001 THE BERGEN'), ('SCH_TYPE', 'Elementary'), ('MANAGED_BY', 1), ('GEO_DISTRI', 15), ('ADMIN_DIST', 15), ('ADDRESS', '309 47 STREET'), ('STATE_CODE', 'NY'), ('ZIP', 11220), ('PRINCIPAL', 'Jennifer Eusanio'), ('PRIN_PH', '718-567-7661'), ('FAX', '718-567-9771'), ('GRADES', 'PK,0K,01,02,03,04,05,SE'), ('City', 'BROOKLYN')])
OrderedDict([('ATS_CODE', '17K002 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'K'), ('BORONUM', 2), ('LOC_CODE', 'K002'), ('SCHOOLNAME', 'M.S. 002'), ('SCH_TYPE', 'Junior High-Intermediate-Middle'), ('MANAGED_BY', 1), ('GEO_DISTRI', 17), ('ADMIN_DIST', 17), ('ADDRESS', '655 PARKSIDE AVENUE'), ('STATE_CODE', 'NY'), ('ZIP', 11226), ('PRINCIPAL', 'ADRIENNE SPENCER'), ('PRIN_PH', '718-462-6992'), ('FAX', '718-284-7717'), ('GRADES', '06,07,08,SE'), ('City', 'BROOKLYN')])
OrderedDict([('ATS_CODE', '21K095 \xa0\xa

OrderedDict([('ATS_CODE', '20K127 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'K'), ('BORONUM', 2), ('LOC_CODE', 'K127'), ('SCHOOLNAME', 'P.S. 127 MCKINLEY PARK'), ('SCH_TYPE', 'Elementary'), ('MANAGED_BY', 1), ('GEO_DISTRI', 20), ('ADMIN_DIST', 20), ('ADDRESS', '7805 7 AVENUE'), ('STATE_CODE', 'NY'), ('ZIP', 11228), ('PRINCIPAL', 'Agatha Alicandro'), ('PRIN_PH', '718-833-2323'), ('FAX', '718-836-9427'), ('GRADES', '0K,01,02,03,04,05,SE'), ('City', 'BROOKLYN')])
OrderedDict([('ATS_CODE', '21K128 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'K'), ('BORONUM', 2), ('LOC_CODE', 'K128'), ('SCHOOLNAME', 'P.S. 128 BENSONHURST'), ('SCH_TYPE', 'Elementary'), ('MANAGED_BY', 1), ('GEO_DISTRI', 21), ('ADMIN_DIST', 21), ('ADDRESS', '2075 84 STREET'), ('STATE_CODE', 'NY'), ('ZIP', 11214), ('PRINCIPAL', 'MARCIA ROBINS'), ('PRIN_PH', '718-373-5900'), ('FAX', '718-266-6254'), ('GRADES', 'PK,0K,01,02,03,04,05,SE'), ('City', 'BROOKLYN')])
OrderedDict([('ATS_CODE', '15K130 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'K'), ('BORONUM', 2), 

OrderedDict([('ATS_CODE', '19K166 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'K'), ('BORONUM', 2), ('LOC_CODE', 'K166'), ('SCHOOLNAME', 'J.H.S. 166 GEORGE GERSHWIN'), ('SCH_TYPE', 'Junior High-Intermediate-Middle'), ('MANAGED_BY', 1), ('GEO_DISTRI', 19), ('ADMIN_DIST', 19), ('ADDRESS', '800 VAN SICLEN AVE'), ('STATE_CODE', 'NY'), ('ZIP', 11207), ('PRINCIPAL', 'MARIA ORTEGA'), ('PRIN_PH', '718-649-0765'), ('FAX', '718-927-2172'), ('GRADES', '06,07,08,SE'), ('City', 'BROOKLYN')])
OrderedDict([('ATS_CODE', '17K167 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'K'), ('BORONUM', 2), ('LOC_CODE', 'K167'), ('SCHOOLNAME', 'P.S. 167 THE PARKWAY'), ('SCH_TYPE', 'Elementary'), ('MANAGED_BY', 1), ('GEO_DISTRI', 17), ('ADMIN_DIST', 17), ('ADDRESS', '1025 EASTERN PARKWAY'), ('STATE_CODE', 'NY'), ('ZIP', 11213), ('PRINCIPAL', 'MARC MARDY'), ('PRIN_PH', '718-774-2640'), ('FAX', '718-953-1954'), ('GRADES', 'PK,0K,01,02,03,04,05,SE'), ('City', 'BROOKLYN')])
OrderedDict([('ATS_CODE', '15K169 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'K

OrderedDict([('ATS_CODE', '15K230 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'K'), ('BORONUM', 2), ('LOC_CODE', 'K230'), ('SCHOOLNAME', 'P.S. 230 DORIS L. COHEN'), ('SCH_TYPE', 'Elementary'), ('MANAGED_BY', 1), ('GEO_DISTRI', 15), ('ADMIN_DIST', 15), ('ADDRESS', '1 ALBEMARLE ROAD'), ('STATE_CODE', 'NY'), ('ZIP', 11218), ('PRINCIPAL', 'SHARON FIDEN'), ('PRIN_PH', '718-437-6135'), ('FAX', '718-871-2624'), ('GRADES', 'PK,0K,01,02,03,04,05,SE'), ('City', 'BROOKLYN')])
OrderedDict([('ATS_CODE', '75K231 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'K'), ('BORONUM', 2), ('LOC_CODE', 'K231'), ('SCHOOLNAME', 'P.S. K231'), ('SCH_TYPE', 'K-8'), ('MANAGED_BY', 1), ('GEO_DISTRI', 20), ('ADMIN_DIST', 75), ('ADDRESS', '5601 16 AVENUE'), ('STATE_CODE', 'NY'), ('ZIP', 11204), ('PRINCIPAL', 'Christina Foti'), ('PRIN_PH', '718-853-1884'), ('FAX', '718-853-5388'), ('GRADES', '0K,01,02,03,04,05,06,07,08,SE'), ('City', 'BROOKLYN')])
OrderedDict([('ATS_CODE', '18K233 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'K'), ('BORONUM', 2), ('LOC_CO

OrderedDict([('ATS_CODE', '13K287 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'K'), ('BORONUM', 2), ('LOC_CODE', 'K287'), ('SCHOOLNAME', 'P.S. 287 BAILEY K. ASHFORD'), ('SCH_TYPE', 'Elementary'), ('MANAGED_BY', 1), ('GEO_DISTRI', 13), ('ADMIN_DIST', 13), ('ADDRESS', '50 NAVY STREET'), ('STATE_CODE', 'NY'), ('ZIP', 11201), ('PRINCIPAL', 'Ms. Michele Rawlins'), ('PRIN_PH', '718-834-4745'), ('FAX', '718-834-6766'), ('GRADES', 'PK,0K,01,02,03,04,05,SE'), ('City', 'BROOKLYN')])
OrderedDict([('ATS_CODE', '21K288 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'K'), ('BORONUM', 2), ('LOC_CODE', 'K288'), ('SCHOOLNAME', 'P.S. 288 THE SHIRLEY TANYHILL'), ('SCH_TYPE', 'K-8'), ('MANAGED_BY', 1), ('GEO_DISTRI', 21), ('ADMIN_DIST', 21), ('ADDRESS', '2950 WEST  25 STREET'), ('STATE_CODE', 'NY'), ('ZIP', 11224), ('PRINCIPAL', 'Joelene-Lynette Kinard'), ('PRIN_PH', '718-449-8000'), ('FAX', '718-449-7682'), ('GRADES', 'PK,0K,01,02,03,04,05,06,07,08,SE'), ('City', 'BROOKLYN')])
OrderedDict([('ATS_CODE', '17K289 \xa0\xa0\xa0\xa0\x

...

OrderedDict([('ATS_CODE', '27Q318 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'Q'), ('BORONUM', 4), ('LOC_CODE', 'Q318'), ('SCHOOLNAME', 'WATERSIDE SCHOOL FOR LEADERSHIP'), ('SCH_TYPE', 'Elementary'), ('MANAGED_BY', 1), ('GEO_DISTRI', 27), ('ADMIN_DIST', 27), ('ADDRESS', '190 BEACH 110 STREET'), ('STATE_CODE', 'NY'), ('ZIP', 11694), ('PRINCIPAL', 'LINDA MUNRO'), ('PRIN_PH', '718-634-1128'), ('FAX', '718-634-1185'), ('GRADES', '06,07,SE'), ('City', 'ROCKAWAY PARK')])
OrderedDict([('ATS_CODE', '27Q319 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'Q'), ('BORONUM', 4), ('LOC_CODE', 'Q319'), ('SCHOOLNAME', 'VILLAGE ACADEMY'), ('SCH_TYPE', 'Elementary'), ('MANAGED_BY', 1), ('GEO_DISTRI', 27), ('ADMIN_DIST', 27), ('ADDRESS', '10-45 NAMEOKE STREET'), ('STATE_CODE', 'NY'), ('ZIP', 11691), ('PRINCIPAL', 'DORIS LEE'), ('PRIN_PH', '718-471-6042'), ('FAX', '718-471-6243'), ('GRADES', '06,07,SE'), ('City', 'QUEENS')])
OrderedDict([('ATS_CODE', '31R861 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'R'), ('BORONUM', 5), ('LOC_CODE', 'R8

OrderedDict([('ATS_CODE', ''), ('BORO', 'O'), ('BORONUM', 0), ('LOC_CODE', 'OAQK'), ('SCHOOLNAME', 'Victory Schools, DBA The Charter School of Excelle'), ('SCH_TYPE', 'Elementary'), ('MANAGED_BY', 2), ('GEO_DISTRI', 0), ('ADMIN_DIST', 0), ('ADDRESS', '260 WARBURTON AVE'), ('STATE_CODE', 'NY'), ('ZIP', 10701), ('PRINCIPAL', ''), ('PRIN_PH', 'N/A'), ('FAX', 'N/A'), ('GRADES', '0K,01,02,03,04'), ('City', 'YONKERS')])
OrderedDict([('ATS_CODE', ''), ('BORO', 'O'), ('BORONUM', 0), ('LOC_CODE', 'OAWF'), ('SCHOOLNAME', 'The Academy Charter School'), ('SCH_TYPE', 'Elementary'), ('MANAGED_BY', 2), ('GEO_DISTRI', 0), ('ADMIN_DIST', 84), ('ADDRESS', '94 Fulton Avenue'), ('STATE_CODE', 'NY'), ('ZIP', 11550), ('PRINCIPAL', ''), ('PRIN_PH', ''), ('FAX', ''), ('GRADES', '0K,01,02,03,04,05'), ('City', 'Hempstead')])
OrderedDict([('ATS_CODE', '28Q342'), ('BORO', 'Q'), ('BORONUM', 4), ('LOC_CODE', 'Q342'), ('SCHOOLNAME', 'The Rochdale Early Advantage Charter School'), ('SCH_TYPE', 'Elementary'), ('MANAGE

OrderedDict([('ATS_CODE', '02M114 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'M'), ('BORONUM', 1), ('LOC_CODE', 'M114'), ('SCHOOLNAME', 'EAST SIDE MIDDLE SCHOOL'), ('SCH_TYPE', 'Junior High-Intermediate-Middle'), ('MANAGED_BY', 1), ('GEO_DISTRI', 2), ('ADMIN_DIST', 2), ('ADDRESS', '331 EAST 91ST STREET'), ('STATE_CODE', 'NY'), ('ZIP', 10128), ('PRINCIPAL', 'DAVID GETZ'), ('PRIN_PH', '212-360-0114'), ('FAX', '212-360-0121'), ('GRADES', '06,07,08,SE'), ('City', 'NEW YORK')])
OrderedDict([('ATS_CODE', '75M721 \xa0\xa0\xa0\xa0\xa0'), ('BORO', 'M'), ('BORONUM', 1), ('LOC_CODE', 'M721'), ('SCHOOLNAME', 'P.S. M721 - MANHATTAN OCCUPATIONAL TRAINING CENTER'), ('SCH_TYPE', 'High school'), ('MANAGED_BY', 1), ('GEO_DISTRI', 2), ('ADMIN_DIST', 75), ('ADDRESS', '250 WEST HOUSTON STREET'), ('STATE_CODE', 'NY'), ('ZIP', 10014), ('PRINCIPAL', 'Antoinette Bello'), ('PRIN_PH', '212-675-7926'), ('FAX', '212-255-3227'), ('GRADES', '09,10,11,12,SE'), ('City', 'MANHATTAN')])
OrderedDict([('ATS_CODE', '28Q206 \xa0\xa0\

OrderedDict([('ATS_CODE', '23K775'), ('BORO', 'K'), ('BORONUM', 3), ('LOC_CODE', 'K775'), ('SCHOOLNAME', 'Leadership Preparatory Ocean Hill Charter School'), ('SCH_TYPE', 'K-8'), ('MANAGED_BY', 2), ('GEO_DISTRI', 23), ('ADMIN_DIST', 84), ('ADDRESS', '51 CHRISTOPHER AVENUE'), ('STATE_CODE', 'NY'), ('ZIP', 11212), ('PRINCIPAL', 'Nikki Bridges'), ('PRIN_PH', '718-250-5767'), ('FAX', '718-250-5768'), ('GRADES', '0K,01'), ('City', 'BROOKLYN')])
OrderedDict([('ATS_CODE', '23K777'), ('BORO', 'K'), ('BORONUM', 3), ('LOC_CODE', 'K777'), ('SCHOOLNAME', 'Ocean Hill Collegiate Charter School'), ('SCH_TYPE', 'Secondary School'), ('MANAGED_BY', 2), ('GEO_DISTRI', 23), ('ADMIN_DIST', 84), ('ADDRESS', '1137 HERKIMER STREET'), ('STATE_CODE', 'NY'), ('ZIP', 11233), ('PRINCIPAL', 'Hannah Soloman'), ('PRIN_PH', '718-250-5765'), ('FAX', '718-250-5766'), ('GRADES', '05'), ('City', 'BROOKLYN')])
OrderedDict([('ATS_CODE', '17K780'), ('BORO', 'K'), ('BORONUM', 3), ('LOC_CODE', 'K780'), ('SCHOOLNAME', 'Brooklyn

OrderedDict([('ATS_CODE', '28Q354'), ('BORO', 'Q'), ('BORONUM', 4), ('LOC_CODE', 'Q354'), ('SCHOOLNAME', 'PS 354'), ('SCH_TYPE', 'Elementary'), ('MANAGED_BY', 1), ('GEO_DISTRI', 28), ('ADMIN_DIST', 28), ('ADDRESS', '126-10 BEDELL STREET'), ('STATE_CODE', 'NY'), ('ZIP', 11434), ('PRINCIPAL', 'Jermaine Green'), ('PRIN_PH', '718-935-3610'), ('FAX', ''), ('GRADES', '01,02,03,04,05,SE'), ('City', 'QUEENS')])
OrderedDict([('ATS_CODE', '32K168'), ('BORO', 'K'), ('BORONUM', 3), ('LOC_CODE', 'K168'), ('SCHOOLNAME', 'Brooklyn School for Math and Research'), ('SCH_TYPE', 'High school'), ('MANAGED_BY', 1), ('GEO_DISTRI', 32), ('ADMIN_DIST', 32), ('ADDRESS', '400 IRVING AVENUE'), ('STATE_CODE', 'NY'), ('ZIP', 11237), ('PRINCIPAL', 'Perry Rainey'), ('PRIN_PH', '718-935-3531'), ('FAX', ''), ('GRADES', '09,10,11,12,SE'), ('City', 'BROOKLYN')])
OrderedDict([('ATS_CODE', '15K684'), ('BORO', 'K'), ('BORONUM', 3), ('LOC_CODE', 'K684'), ('SCHOOLNAME', 'Millennium High School'), ('SCH_TYPE', 'High school'),

In [ ]:

from dbfread import DBF
import pandas as pd


###########OPEN data FILE###################

   
#opening and printing  data from csv
i=0
with open('randomized_edu_data_Nov_2018.csv', 'r') as csvFile:
    reader = csv.reader(csvFile)
    for row in reader:
        i = i+1#print(row)
csvFile.close()        
print('number of rows in data randomize schools', i)        




"""
We create dataframe object from csv file with datapoints around the world file files put in pandas format with header
"""
data = pd.read_csv("randomized_edu_data_Nov_2018.csv", error_bad_lines=False, delim_whitespace =True)#,names = col_H)


print('data preprocessing')
print('example of randomized data')
data.head()
print(data.iloc[0])
print(data.iloc[1])

print('plotting the basemap')
plot_basemap()



#we first do data analysis of the csv file we uploaded

print('extracting column from data table')
Column_tags = data.set_index('schools', drop = False)

N_schools = data['schools'].nunique() #estimate number of various schools
N_schools = data['country'].nunique() #estimate number of different countries

print('number of different categories')
#N_tags = len(array_unique_tags_df(df_all))
print(N_schools)

#then we can do some data visualisation of educational institutions
print('plotting countries colors according to the number of educational institutions in the map')
#highlight_country(country)
# then plot some countries on with color encoded in estimated_trips
for index, row in data_2014.iterrows(): 
    print (row["source_iso3"], row["estimated_trips"])
    plotCountryPatch(ax2, row["source_iso3"], row["estimated_trips"])




number of rows in data randomize schools 92


b'Skipping line 117: expected 1 fields, saw 31\nSkipping line 118: expected 1 fields, saw 3\n'


data preprocessing
example of randomized data
Latitude,Longitude,Name,Street,City,State,Country,Zipcode,Color,Photo;URL,Description,Email,Phone,Website;Details;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;    48.135125,11.581981,Maths-physics-literature;c...
Name: 0, dtype: object
Latitude,Longitude,Name,Street,City,State,Country,Zipcode,Color,Photo;URL,Description,Email,Phone,Website;Details;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;    German,;Russian;languages;;;;;;;;;;;;;;;;;;;;;...
Name: 1, dtype: object
plotting the basemap
plotting basemap
extracting column from data table


There is another way to visualise data points on a map using folium map.

One can see here 
https://python-graph-gallery.com/313-bubble-map-with-folium/

In [41]:
# plotting map with folium map
import folium
import pandas as pd
 
# Make a data frame with dots to show on the map
data = pd.DataFrame({
   'lat':[-58, 2, 145, 30.32, -4.03, -73.57, 36.82, -38.5],
   'lon':[-34, 49, -38, 59.93, 5.33, 45.52, -1.29, -12.97],
   'name':['Buenos Aires', 'Paris', 'melbourne', 'St Petersbourg', 'Abidjan', 'Montreal', 'Nairobi', 'Salvador'],
   'value':[10,12,40,70,23,43,100,43]
})
data
 
# Make an empty map
m = folium.Map(location=[20,0], tiles="Mapbox Bright", zoom_start=2)
 
# I can add marker one by one on the map
for i in range(0,len(data)):
   folium.Circle(
      location=[data.iloc[i]['lon'], data.iloc[i]['lat']],
      popup=data.iloc[i]['name'],
      radius=data.iloc[i]['value']*10000,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(m)

#display the map 
#m

# Save it as html
#m.save('mymap.html')
#m.save('312_markers_on_folium_map1.html')#plt.show()

In [39]:
'''
We plot location of datapoints on Folium map.

Input:
latitude and longitude from dataframe moves_dataframe
  moves_dataframe = pd.DataFrame(data={
        'date': moves_processed_data['date'],
        'steps': moves_processed_data['steps'],
        'distance': moves_processed_data['distance'],
        'latitude': moves_processed_data['lat'],
        'longitude': moves_processed_data['lon']
    }) 

Output:
Folium map with datapoints
'''

import folium


#first we get datapoints from the data
latitude = data.latitude.values
longitude = data.longitude.values


#then we zip two arrays of lat, lon of datapoints
latlon = list(zip(latitude, longitude))
#print(latlon)

#We put map to show first location of New York 40.7808 et la longitude de la ville de New York City est -73.9772.

mapit = folium.Map( location=[40.7808 , -73.9772], zoom_start=6 )
for coord in latlon:
    folium.Marker( location=[ coord[0], coord[1] ], fill_color='#43d9de', radius=8 ).add_to( mapit )
mapit
